In [ ]:
%matplotlib inline


Autograd: automatic differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.

Variable
--------

``autograd.Variable`` is the central class of the package. It wraps a
Tensor, and supports nearly all of operations defined on it. Once you
finish your computation you can call ``.backward()`` and have all the
gradients computed automatically.

You can access the raw tensor through the ``.data`` attribute, while the
gradient w.r.t. this variable is accumulated into ``.grad``.

<img src="tutorial_img/variable.png">

   Variable

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Variable`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each variable has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Variable`` (except for Variables created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Variable``. If ``Variable`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.



As you perform operations on a Variable object, Pytorch will create a computation graph that remembers the past inputs and actions you took. See the example graph below
<img src="tutorial_img/comp_graph.png" width="50%">

The inputs are $a$ and $b$. $c$ and $d$ are intermediate values, and $e$ is the output. Let's see what this looks like in code! Note that all of the nodes we will create will be $\texttt{Variable}$ objects, so if you want to extract the actual value, you need to call something like $\texttt{a.data[0]}$.

In [ ]:
import torch
from torch.autograd import Variable

First we create the $a$ and $b$ nodes. Since they are inputs, we initialize them as $\texttt{Variable}$. This creates the bottom two nodes in the graph above.

In [ ]:
a = Variable(torch.FloatTensor([2]), requires_grad=True)
b = Variable(torch.FloatTensor([1]), requires_grad=True)
print('a=%s, b=%s' % (a.data[0], b.data[0]))

Now we create the intermediate nodes $c$ and $d$

In [ ]:
c = a + b
d = b + 1
print('c=%s, d=%s' % (c.data[0], d.data[0]))

Finally, we calculate the output $e = cd$

In [ ]:
e = c * d
print(e.data[0])

Now, if we want to calculate the derivatives, $\frac{\partial e}{\partial a}$ and $\frac{\partial e}{\partial b}$, we just call $\texttt{e.backward()}$. Are the values outputted below what you would expect?

<img src="tutorial_img/comp_graph_d.png" width="50%">

Remember that we had $c = a+b$, $d = b+1$, and $e = c*d$. In order the calculate $\frac{\partial e}{\partial a}$ and $\frac{\partial e}{\partial b}$, we need to use the chain rule in calculus.

Write your answers for problems 1 to 3 in terms of $a, b, c, d, e$

## Problem 1
What are the partial derivatives $\Large\frac{\partial e}{\partial c}$, $\Large\frac{\partial e}{\partial d}$?

$\Large\frac{\partial e}{\partial c} = $

$\Large\frac{\partial e}{\partial d} = $

## Problem 2
What are the partial derivatives $\Large\frac{\partial c}{\partial a}$, $\Large\frac{\partial c}{\partial b}$, $\Large\frac{\partial d}{\partial b}$? 

$\Large\frac{\partial c}{\partial a} = $

$\Large\frac{\partial c}{\partial b} = $

$\Large\frac{\partial d}{\partial b} = $

## Problem 3
Now use the chain rule and your answers from previous parts to find $\Large\frac{\partial e}{\partial a}$ and $\Large\frac{\partial e}{\partial b}$. Note that the gradient for $b$ is coming in from two different signals.

$\Large\frac{\partial e}{\partial a} = $

$\Large\frac{\partial e}{\partial b} = $

## Problem 4
Now, what are $\Large\frac{\partial e}{\partial a}$ and $\Large\frac{\partial e}{\partial b}$ given our previous inptus $a=2$, $b=1$?

$\Large\frac{\partial e}{\partial a} = $

$\Large\frac{\partial e}{\partial b} = $

See if your calculations match the output below!

In [ ]:
e.backward()

In [ ]:
print('de/da', a.grad.data[0])
print('de/db', b.grad.data[0])

This is exactly what autograd is doing! It is constructing a computational graph that uses chain rule to calculate the derivates of the output with respect to all of its inputs!

## Example
Here is another example with the input as a $2\times 2$ matrix, where $x = \begin{bmatrix}x_1&x_2\\x_3&x_4\end{bmatrix} = \begin{bmatrix}1&1\\1&1\end{bmatrix}$. And our computational graph is modelling the function $3(x+2)^2$, where the square is a elementwise square of the matrix. More specifically, we have
$$\begin{bmatrix}3(x_1+2)^2&3(x_2+2)^2\\3(x_3+2)^2&3(x_4+2)^2\end{bmatrix}$$

In [ ]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

Do an operation of variable:



In [ ]:
y = x + 2
print(y)

``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [ ]:
print(y.grad_fn)

Do more operations on y



In [ ]:
z = y * y * 3
out = z.mean()

print(z, out)

Gradients
---------
let's backprop now
``out.backward()`` is equivalent to doing ``out.backward(torch.Tensor([1.0]))``



In [ ]:
out.backward()

print gradients d(out)/dx




In [ ]:
print(x.grad)

You should have got a matrix of ``4.5``. Let’s call the ``out``
*Variable* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



You can do many crazy things with autograd!



In [ ]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

In [ ]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

## Exercises

$\textbf{Instructions}$: Complete the exercises below. Make sure to only use basic operations (+, -, *, etc) or torch functions when doing tensor calculations

### Question 1

Create computational graph with $x$ as input, and output as $max(x, 0)$. This is the ReLU activation which is commonly used instead of the sigmoid in neural nets!

In [ ]:
def q1(x):
    zero = Variable(torch.FloatTensor([0])) # use this as the zero in max(x, 0)
    """ Your Code Here """
    return None
    
def check_q1():
    x1 = Variable(torch.FloatTensor([-1]), requires_grad=True)
    x2 = Variable(torch.FloatTensor([1]), requires_grad=True)
    assert q1(x1).data[0] == 0
    assert q1(x2).data[0] == 1
    print('Passed')
check_q1()

Now, write a function that prints out the gradient of $max(x,0)$ at the given $x$. You may need to call your function $\texttt{q1(x)}$

In [ ]:
def grad_q1(x):
    """ Your Code Here """

def check_grad_q1():
    x1 = Variable(torch.FloatTensor([-10]), requires_grad=True)
    x2 = Variable(torch.FloatTensor([100]), requires_grad=True)
    assert grad_q1(x1).data[0] == 0
    assert grad_q1(x2).data[0] == 1
    print('Passed')
check_grad_q1()

## Question 2
Find the derivative of the function $f(x, y, z) = \frac{(x-1)(y-2)^2(y-3)^3}{\sqrt{x+y+z}}$ with respect to $x, y$, and $z$

In [ ]:
def q2(x, y, z): # The inputs are all torch Variable objects
    # 1. Calculate f(x,y,z)
    """ Your Code Here """
    # 2. Find and return df/dx, df/dy, df/dz as x, y, and z (in that order)
    """ Your Code Here """
    return None

def check_q2():
    import numpy as np
    x = Variable(torch.FloatTensor([4]), requires_grad=True)
    y = Variable(torch.FloatTensor([5]), requires_grad=True)
    z = Variable(torch.FloatTensor([6]), requires_grad=True)
    
    x_grad, y_grad, z_grad = q2(x, y, z)
    x_grad, y_grad, z_grad = x_grad.data[0], y_grad.data[0], z_grad.data[0]
    
    assert np.allclose(x_grad, 56.46809387207031)
    assert np.allclose(y_grad, 119.21041870117188)
    assert np.allclose(z_grad, 181.95274353027344)
    
    print('Passed')
    
check_q2()

## Question 3
Find the derivative of the perceptron with respect to each of its inputs in $x$ (The perceptron is the same thing as the neuron you created in the NeuralNetwork notebook, just this time with torch objects and gradients). There is no activation function. $\textbf{Important: Use torch.randn to generate your weights and bias.}$

Make sure you first generate your weights (vector) $\textit{and then}$ bias (scalar), or else you might fail to test even though you have a correct implementation

In [ ]:
# x is a 5 dimensional vector Variable object (no need to create a Variable, it will be already created)
def q3(x):
    # 1. Calculate f(x)
    """ Your Code Here """
    # 2. Find and return df/dx, df/dw, df/b - Which ones of these are vectors? (Make sure you know/understand this)
    # Remember that if you want to derivative wrt w, b, they need to be Variable objects
    """ Your Code Here """
    return None

def check_q3():
    import numpy as np
    torch.manual_seed(0)
    x = Variable(torch.randn(5), requires_grad=True)
    x_grad, w_grad, b_grad = q3(x)
    x_grad, w_grad, b_grad = x_grad.data, w_grad.data, b_grad.data
    x_grad, w_grad, b_grad = x_grad.numpy(), w_grad.numpy(), b_grad.numpy()
    
    xg_correct = [-1.3985955, 0.40334684, 0.83802634, -0.7192576, -0.40334353]
    wg_correct = [1.5409961, -0.2934289, -2.1787894, 0.56843126, -1.0845224]
    bg_correct = 1
    assert np.allclose(x_grad, xg_correct)
    assert np.allclose(w_grad, wg_correct)
    assert np.allclose(b_grad, bg_correct)
    print('Passed')
    
    
check_q3()